In [1]:
import numpy as np
import pandas as pd
import category_encoders as ce
import os, gc, pickle, time, datetime
from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

SEED = 42
np.random.seed(SEED)

---
### Load Data

In [73]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings                    
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
    mem_usg = df.memory_usage().sum() / 1024**2 
    return df, NAlist

def extract_id_meter(df, building_id, meter):
    temp = df[df['building_id'] == building_id]
    temp = temp[temp['meter'] == meter]
    return temp

# Prepare data

In [72]:
def prep_core_data(df):
    # Check lossed Date  ####################################################################
    id_list = []
    meter_list = []
    rows_list = []

    for id_ in range(df['building_id'].nunique()):
        for meter in range(4):
            temp = extract_id_meter(df, id_, meter)
            rows = temp.shape[0]
            if rows not in [0, 8784]:
                id_list.append(id_)
                meter_list.append(meter)
                rows_list.append(rows)

    df_loss = pd.DataFrame({
        'building_id': id_list,
        'meter': meter_list,
        'rows': rows_list
    })
    del id_list, meter_list, rows_list
    
    # Fill dropped Date
    def fill_date(_df, building_id, meter):
        temp = extract_id_meter(_df, building_id, meter)

        dates_DF = pd.DataFrame(pd.date_range('2016-1-1', periods=366*24, freq='H'), columns=['Date'])
        dates_DF['Date'] = dates_DF['Date'].apply(lambda x: x.strftime('%Y-%m-%d %T'))

        temp = pd.merge(temp, dates_DF, how="outer", left_on=['timestamp'], right_on=['Date'])
        del temp['timestamp']
        temp = temp.rename(columns={'Date': 'timestamp'})
        temp['building_id'] = building_id
        temp['meter'] = meter

        temp = temp[temp['meter_reading'].isnull()]
        _df = pd.concat([_df, temp], axis=0, ignore_index=True)

        return _df

    for _id, meter in zip(df_loss['building_id'], df_loss['meter']):
        df = fill_date(df, _id, meter)
        
    # Interpolate    ####################################################################
    for _id in range(df['building_id'].nunique()):
        for meter in df['meter'].unique().tolist():
            # building_id, meterで抽出
            temp = extract_id_meter(df, _id, meter)
            temp = temp.sort_values(by='timestamp')
            
            if temp.empty:
                continue
            # meter_readingが0のものを欠損として扱う
            temp.loc[temp['meter_reading'] == 0, 'meter_reading'] = np.nan
            # 欠損を内挿で埋める
            temp['meter_reading'] = temp['meter_reading'].interpolate(limit_area='inside', limit=5)
            df.loc[temp.index, 'meter_reading'] = temp.loc[temp.index, 'meter_reading']
            
    # Dropna    ####################################################################
    df.dropna(inplace=True)
            
    return df

In [4]:
# Preprocessing Weather Data
def prep_weather_data(df):
    # Drop Features  #####################################################################
    drop_col = ['precip_depth_1_hr', 'sea_level_pressure', 'cloud_coverage']
    df.drop(drop_col, axis=1, inplace=True)
    
    # Create Features per Site Id  #####################################################################
    # Fillna(Interpolate)
    for i in range(df['site_id'].nunique()):
        temp = df[df['site_id'] == i]
        temp = temp.sort_values(by='timestamp')

        # Interpolation
        cols = ['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']
        for c in cols:
            temp[c] = temp[c].interpolate(limit_direction='both')
            df.loc[temp.index, c] = temp.loc[temp.index, c]
                
                
    # relative Hummd  #####################################################################
    # https://soudan1.biglobe.ne.jp/qa5356721.html
    a_temp = df['air_temperature'].values
    d_temp = df['dew_temperature'].values
    def SaturatedWaterVaporPressure(values):
        return 6.11 * 10 ** (7.5 * values / (237.3 + values))
    
    a_temp = SaturatedWaterVaporPressure(a_temp)
    d_temp = SaturatedWaterVaporPressure(d_temp)
    
    df['relative_hummd'] = d_temp / a_temp * 100
    del a_temp, d_temp
    
    # Wind Direction  #####################################################################
    df.loc[df['wind_direction'] == 65535, 'wind_direction'] = np.nan
    df['wind_direction'] = np.radians(df['wind_direction'])
    df['wind_direction_sin'] = np.sin(df['wind_direction'])
    df['wind_direction_cos'] = np.cos(df['wind_direction'])
    df['wind_direction_tan'] = np.tan(df['wind_direction'])
    
    df['wind_speed_sin'] = df['wind_speed'] * df['wind_direction_sin']
    df['wind_speed_cos'] = df['wind_speed'] * df['wind_direction_cos']
    
    
    # Create Features per Site Id  #####################################################################
    for i in range(df['site_id'].nunique()):
        temp = df[df['site_id'] == i]
        temp = temp.sort_values(by='timestamp')
    # Rolling
        cols = ['air_temperature', 'dew_temperature', 'relative_hummd', 'wind_speed_sin', 'wind_speed_cos']
        for c in cols:
            for window in range(2, 5, 1):
                colname = '{}_roll_{}_mean'.format(c, window)
                temp[colname] = temp[c].rolling(window).mean()
                df.loc[temp.index, colname] = temp.loc[temp.index, colname]
                colname = '{}_roll_{}_sum'.format(c, window)
                temp[colname] = temp[c].rolling(window).sum()
                df.loc[temp.index, colname] = temp.loc[temp.index, colname]

        # Shift
        cols = ['air_temperature', 'dew_temperature', 'relative_hummd', 'wind_speed_sin', 'wind_speed_cos']
        for c in cols:
            for period in range(1, 3, 1):
                colname = '{}_shift_{}'.format(c, period)
                shifted = temp[c].shift(periods=period)
                temp[colname] = temp[c] - shifted
                df.loc[temp.index, colname] = temp.loc[temp.index, colname]
                      
    return df
    

In [5]:
class PreprocessingDataset:
    def __init__(self):
        self.df = None
        
    def prep(self, df, df_weather, df_building, mode='train'):
        
        # Core Data Prep  #####################################################################
        if mode == 'train':
            df = prep_core_data(df)
            
        # Weather Data Prep  #####################################################################
        df_weather = prep_weather_data(df_weather)
        
        # merge data  #####################################################################
        df = pd.merge(df, df_building, how="left", on=["building_id"])
        df = pd.merge(df, df_weather, how='left', on=["site_id", "timestamp"])
        self.df, _ = reduce_mem_usage(df)
        del df, df_weather, df_building
        gc.collect()
        
        # Datetime  #####################################################################
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['month'] = self.df['timestamp'].dt.month.astype(np.uint8)
        self.df['day'] = self.df['timestamp'].dt.day.astype(np.uint8)
        self.df['hour'] = self.df['timestamp'].dt.hour.astype(np.uint8)
        self.df['weekday'] = self.df['timestamp'].dt.weekday.astype(np.uint8)
        # Sort Timestamp  #####################################################################
        self.df = self.df.sort_values(by='timestamp', ascending=True).reset_index(drop=True)
        del self.df['timestamp']
        gc.collect()
        
        # Year Built  #####################################################################
        self.df['year_built'] = self.df['year_built'] - 1900
        
        # square_feet  #####################################################################
        self.df['square_feet'] = np.log(self.df['square_feet'])
        
        # LabelEncoder  #####################################################################
        list_cols = ['primary_use']
        if mode == 'train':
            self.ce_oe = ce.OrdinalEncoder(cols=list_cols,handle_unknown='impute')
            self.df = self.ce_oe.fit_transform(self.df)
        elif mode == 'test':
            self.df = self.ce_oe.transform(self.df)
        
        # Data Type  #####################################################################
        # float32
        cols = self.df.select_dtypes(np.float64).columns
        for c in cols:
            self.df[c] = self.df[c].astype(np.float32)
        # category
        cols = ["site_id", "building_id", "primary_use", "hour", "day", "weekday", "month", "meter"]
        for c in cols:
            self.df[c] = self.df[c].astype('category')
            
        # sort row_id  #####################################################################
        if mode == 'test':
            self.df = self.df.sort_values(by='row_id').reset_index(drop=True)


---
# Build Model

In [6]:
class Trainer:
    
    def __init__(self):
        pass
    
    def train(self, df, params, cv, num_boost_round, early_stopping_rounds, verbose, split=None):
        self.y = np.log1p(df['meter_reading'])
        self.x = df.drop(['meter_reading'], axis=1)
        self.cv = cv
        self.oof = 0.0
        self.models = []
        self.features = self.x.columns
        
        if split is None:
            _cv = cv.split(self.x)
        else:
            _cv = cv.split(self.x, self.x[split])
        
        for i, (trn_idx, val_idx) in enumerate(_cv):
            print('Fold {} Model Creating...'.format(i+1))
            _start = time.time()

            train_data = lgb.Dataset(self.x.iloc[trn_idx], label=self.y.iloc[trn_idx])
            val_data = lgb.Dataset(self.x.iloc[val_idx], label=self.y.iloc[val_idx], reference=train_data)

            model = lgb.train(params, 
                              train_data, 
                              num_boost_round=num_boost_round,
                              valid_sets=(train_data, val_data),
                              early_stopping_rounds=early_stopping_rounds,
                              verbose_eval=verbose)

            y_pred = model.predict(self.x.iloc[val_idx], num_iteration=model.best_iteration)
            error = np.sqrt(mean_squared_error(y_pred, self.y.iloc[val_idx]))
            self.oof += error / cv.n_splits

            print('Fold {}: {:.5f}'.format(i+1, error))

            elapsedtime = time.time() - _start
            print('Elapsed Time: {}'.format(str(datetime.timedelta(seconds=elapsedtime))))
            print('')
        
            self.models.append(model)
        print('OOF Error: {:.5f}'.format(self.oof))
        
        return model
        
    def predict(self, df, step_size=500):
        
        if 'row_id' in df.columns:
            df.drop('row_id', axis=1, inplace=True)
            
        i=0
        res=[]
        for j in range(int(np.ceil(df.shape[0]/step_size))):
            res.append(np.expm1(sum([model.predict(df.iloc[i:i+step_size], num_iteration=model.best_iteration) for model in self.models]) / self.cv.n_splits))
            i+=step_size
            
        res = np.concatenate(res)
        
        return res
    
    def get_feature_importance(self):
        importance = np.zeros(len(self.features))
        
        for i in range(len(self.models)):
            importance += self.models[i].feature_importance() / len(self.models)
        
        importance_df = pd.DataFrame({
            'feature': self.features,
            'importance': importance
        })
        importance_df = importance_df.sort_values(by='importance', ascending=False)

        fig = plt.figure(figsize=(12, 20))
        sns.barplot(x='importance', y='feature', data=importance_df)
        plt.show()

In [ ]:
%%time
# Prep Train Data  #####################################################################
print('Data Loading...')
train = pd.read_csv("../input/train.csv")
df_weather_train = pd.read_csv("../input/weather_train.csv")
df_building = pd.read_csv("../input/building_metadata.csv")

# Sampling
train = train.sample(frac=0.01, random_state=SEED)

data = PreprocessingDataset()
data.prep(train, df_weather_train, df_building, mode='train')
del train, df_weather_train, df_building
print('Data Already...')

data.df.head(100).to_csv('../Output/Prep_train.csv', index=False)

# Config  #####################################################################
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse'},
    'subsample': 0.7,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9
}

num_folds = 4
cv = StratifiedKFold(num_folds, shuffle=True, random_state=42)
num_boost_round = 6000
early_stopping_rounds = 100
verbose = 1000
split = 'building_id'

# Model Create  #####################################################################
model = Trainer()
_ = model.train(data.df, params, cv, num_boost_round, early_stopping_rounds, verbose, split)

# Plot Feature Importances  #####################################################################
model.get_feature_importance()

Data Loading...
Data Already...
Fold 1 Model Creating...
Training until validation scores don't improve for 100 rounds


In [ ]:
# Chunksize ver

chunk_size = 5000
test_reader = pd.read_csv("../input/test.csv", chunksize=chunk_size)
df_weather_test = pd.read_csv("../input/weather_test.csv")
df_building = pd.read_csv("../input/building_metadata.csv")

pred_all = []

for test in tqdm(test_reader):
    data.prep(test, df_weather_test, df_building, mode='test')
    pred = model.predict(data.df)
    pred_all.append(pred)
    
pred_all = np.concatenate(pred_all)

# Make Submission File
sub = pd.read_csv("../input/sample_submission.csv")
sub["meter_reading"] = pred
today = datetime.datetime.now().strftime('%Y%m%d')
sub.to_csv("../Output/submission_{}_oof_{:.3f}.csv".format(today, model.oof), index=False)

In [10]:
test = pd.read_csv("../input/test.csv")
df_weather_test = pd.read_csv("../input/weather_test.csv")
df_building = pd.read_csv("../input/building_metadata.csv")

data.prep(test, df_weather_test, df_building, mode='test')
del test, df_weather_test, df_building
gc.collect()

pred = model.predict(data.df)

# Make Submission File
sub = pd.read_csv("../input/sample_submission.csv")
sub["meter_reading"] = pred
today = datetime.datetime.now().strftime('%Y%m%d')
sub.to_csv("../Output/submission_{}_oof_{:.3f}.csv".format(today, model.oof), index=False)

MemoryError: Unable to allocate array with shape (50, 41697600) and data type float64

In [ ]:
train = pd.read_csv("../input/train.csv")
print(train.isnull().sum())
print(train.shape)

train = prep_core_data(train)

print(train.isnull().sum())
print(train.shape)


building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64
(20216100, 4)


d:\python\kaggle\ashrae~1\venv~1\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [55]:
_train = train.copy().reset_index(drop=True)

for _id in tqdm(range(_train['building_id'].nunique())):
    for meter in _train['meter'].unique().tolist():

        temp = _train[_train['building_id'] == _id]
        temp = temp[temp['meter'] == meter]
        temp = temp.sort_values(by='timestamp')
        
        if temp.empty:
            continue

        temp['meter_reading'] = temp['meter_reading'].interpolate(limit_area='inside')
        _train.loc[temp.index, 'meter_reading'] = temp.loc[temp.index, 'meter_reading']

In [56]:
print(_train.isnull().sum())

building_id           0
meter                 0
meter_reading    186582
timestamp             0
dtype: int64


In [45]:
train.isnull().sum()

building_id           0
meter                 0
meter_reading    689820
timestamp             0
dtype: int64

In [57]:
a = _train[_train['building_id'] == 7]
a = a[a['meter'] == 1]

In [58]:
a = a.sort_values(by='timestamp')

In [71]:
a[1420:1430]

,building_id,meter,meter_reading,timestamp
6327470,7,1,4938.61,2016-04-28 13:00:00
6329814,7,1,5149.66,2016-04-28 14:00:00
6332161,7,1,5107.45,2016-04-28 15:00:00
6334506,7,1,4854.19,2016-04-28 16:00:00
6336851,7,1,4558.72,2016-04-28 17:00:00
6339197,7,1,4769.77,2016-04-28 18:00:00
6341541,7,1,4643.14,2016-04-28 19:00:00
6343887,7,1,4347.67,2016-04-28 20:00:00
6346230,7,1,3630.09,2016-04-28 21:00:00
6348574,7,1,2574.83,2016-04-28 22:00:00


In [70]:
a.dropna(inplace=True)